# Training the fast face

In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
import glob
import os
# import tensorflow.contrib.slim as slim
import sys
from tensorflow import keras

In [2]:
sys.path.append('../')

In [3]:
import anchors as Anchors
import backbone as Backbone
import model as Model
import dataprocess as DataHandler
import loss as Losses

# Contents
1. Getting train and validation data in required format
2. Loading model
3. Setting training parameters: batch size, optimizer, lr, and other parameters
4. Get the loss function 
5. Writing the training process
6. Backpropagating 
7. Storing weights and testing 

## Getting train data 

In [4]:
image_folder_path = "../../dataset/Wider/WIDER_train/images/"
label_path = "../../dataset/Wider/wider_face_split/wider_face_train_bbx_gt.txt"

In [5]:
widerDataset = DataHandler.WiderDataset(image_folder_path,label_path)

In [6]:
widerDataset.get_item(0)[1]

[[48, 130, 124, 264],
 [240, 194, 300, 300],
 [382, 156, 440, 230],
 [504, 138, 576, 234],
 [802, 142, 864, 226]]

## Loading Model

In [7]:
anchors_cfg_1 = {'base_size':16,'ratios':[1],'scales':2 ** np.arange(0, 1),'stride':1}
anchors_cfg = {}
anchors_cfg[0] = {'base_size':16,'ratios':[1],'scales':2 ** np.arange(0, 1),'stride':4}
anchors_cfg[1] = {'base_size':16,'ratios':[1],'scales':2 ** np.arange(0, 1),'stride':8}
anchors_cfg[2] = {'base_size':16,'ratios':[1],'scales':2 ** np.arange(0, 1),'stride':16}
anchors_cfg[3] = {'base_size':16,'ratios':[1],'scales':2 ** np.arange(0, 1),'stride':32}
anchors_cfg[4] = {'base_size':16,'ratios':[1],'scales':2 ** np.arange(0, 1),'stride':64}
input_shape = (480,640,3)#(480,640,3)

In [8]:
model,anchors = Model.resnet50_retinanet(input_shape=input_shape,anchors_cfg=anchors_cfg)

Instructions for updating:
Colocations handled automatically by placer.


/home/aditya0212jain/anaconda3/envs/work/lib/python2.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


fpn0: (60,80)
fpn1: (30,40)
fpn2: (15,20)
fpn3: (8,10)
fpn4: (4,5)


## compiling with learning rate, loss and optimizer

In [9]:
epochs = 2
batch_size = 32
lr = 0.01
model.compile(
    loss={
        'regression': Losses.smooth_l1(),
        'classification': Losses.focal()
    },
    optimizer=keras.optimizers.Adam(lr=lr, clipnorm=0.001)
)

ValueError: Unknown entry in loss dictionary: "regression". Only expected the following keys: ['out']

## Testing on sample image

In [9]:
im = Image.open('../../dataset/images/2019-06-13 16_25_53__000497__91b35108-8da2-11e9-827b-cf5f9d5a17ad.jpg')
im1 = np.array(im,dtype=np.float32)
im1 = np.expand_dims(im1,axis=0)
print("image shape: ",im1.shape)
import time

('image shape: ', (1, 480, 640, 3))


In [10]:
a = time.time()
b = model.predict(im1)
c = time.time()

In [11]:
c - a

10.786702871322632

In [14]:
b.shape

(1, 24576800, 5)

In [14]:
anchors[4]

array([[   0.,    0.,   15.,   15.],
       [  64.,    0.,   79.,   15.],
       [ 128.,    0.,  143.,   15.],
       [ 192.,    0.,  207.,   15.],
       [ 256.,    0.,  271.,   15.],
       [   0.,   64.,   15.,   79.],
       [  64.,   64.,   79.,   79.],
       [ 128.,   64.,  143.,   79.],
       [ 192.,   64.,  207.,   79.],
       [ 256.,   64.,  271.,   79.],
       [   0.,  128.,   15.,  143.],
       [  64.,  128.,   79.,  143.],
       [ 128.,  128.,  143.,  143.],
       [ 192.,  128.,  207.,  143.],
       [ 256.,  128.,  271.,  143.],
       [   0.,  192.,   15.,  207.],
       [  64.,  192.,   79.,  207.],
       [ 128.,  192.,  143.,  207.],
       [ 192.,  192.,  207.,  207.],
       [ 256.,  192.,  271.,  207.]])

In [22]:
key  = 0

In [23]:
po = Anchors.generate_reference_anchors(base_size=anchors_cfg[key]['base_size'],
                                                     ratios=anchors_cfg[key]['ratios'],
                                                     scales=anchors_cfg[key]['scales'])

In [17]:
b.shape

(1, 6800, 5)

## Setting training parameters
### Creating generator

In [58]:
d = widerDataset.get_item(34)

In [59]:
dp = np.array(d[1])

In [60]:
dp.shape

(1, 4)

In [65]:
x = {}
for j in range(5):
    x[j] = []
    tv = Anchors.get_labels_and_regression_values(anchors[j],dp,positive_threshold=0.01)
    for i in range(len(tv[0])):
        if tv[0][i][4]==1:
            x[j].append(tv[0][i])

In [66]:
x

{0: [], 1: [], 2: [], 3: [], 4: []}

In [57]:
an[1] = {'s':34}

NameError: name 'an' is not defined

In [24]:
an[45] = {'t':90}

In [25]:
an[34] = {'p':34}

In [26]:
sorted(an.keys())

[0, 1, 34, 45]

In [28]:
a = [23,45,5,6,77,0]

In [32]:
b = sorted(a)

In [33]:
b

[0, 5, 6, 23, 45, 77]